In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.mode.chained_assignment = None #for not showing warning message 
pd.options.display.float_format = '{:,.4f}'.format #for setting thhe float format to this notebook

# Importing Master Table 

In [3]:
jccycle                   =      pd.read_excel("D:\Analytics\Marketing Report\Journey Cycle\Master Journey Cycle.xlsx")
product_hierarchy         =      pd.read_excel("D:\Analytics\Marketing Report\FY 19-20\Master Data\Product Hierarchy.xlsx")
static_category_lookup    =      pd.read_excel("D:\Analytics\Marketing Report\FY 19-20\Master Data\Static_Category_Lookup.xlsx")

# Importing JC Table

In [4]:
# JC 9
data9_keyac = pd.read_excel("D:\Analytics\Marketing Report\FY 19-20\JC 09\JC09-Key Ac's.xlsb",engine='pyxlsb')
data9_nonsouth = pd.read_excel("D:\Analytics\Marketing Report\FY 19-20\JC 09\JC09-Non South.xlsb",engine = 'pyxlsb')
data9_south = pd.read_excel("D:\Analytics\Marketing Report\FY 19-20\JC 09\JC09-South.xlsb",engine = 'pyxlsb')

In [59]:
# temp1 = data9_keyac.isnull().sum().to_frame().reset_index()
# temp1[temp1[0] != 0].reset_index()

In [60]:
# temp2 = data9_nonsouth.isnull().sum().to_frame().reset_index()
# temp2[temp2[0] != 0].reset_index()

In [61]:
# temp3 = data9_south.isnull().sum().to_frame().reset_index()
# temp3[temp3[0] != 0].reset_index()

In [5]:
data9 =  data9_keyac.append(data9_nonsouth)
data9 = data9.append(data9_south)

In [6]:
temp4 = data9.isnull().sum().to_frame().reset_index()
temp4[temp4[0] != 0]

,index,0
2,DistName,8052
3,DistType,8052
4,Rtrid,8052
5,RtrCode,11094
6,RtrName,11094
7,RtrGroupCde,11094
9,BillNo,8053
10,InvDate,8052
13,PrdName,8052
16,NetAmt,8052


In [93]:
data9.columns

Index(['Distcode', 'RS CODE', 'DistName', 'DistType', 'Rtrid', 'RtrCode',
       'RtrName', 'RtrGroupCde', 'RET GRP', 'BillNo', 'InvDate', 'PERIOD',
       'PrdCode', 'PrdName_x', 'BRAND CAT', 'TotQty', 'NetAmt', 'BaseValue',
       'In Lakhs', 'REGION', 'ZONE', 'PrdName_y', 'Brand', 'RB', 'Channel',
       'Div', 'BrdCat', 'BrdPk', 'PowerSKU', 'Old_Category', 'New_Category'],
      dtype='object')

In [35]:
data9.head(2)

,Distcode,RS CODE,DistName,DistType,Rtrid,RtrCode,RtrName,RtrGroupCde,RET GRP,BillNo,...,PERIOD,PrdCode,PrdName,BRAND CAT,TotQty,NetAmt,BaseValue,In Lakhs,REGION,ZONE
0,MT0600032000290003200029,3200029,VENKATA SAI SALES CORPORATION,Key A/c Distributor,48.0000,RET0048,AVENUE SUPERMARTS LTD RAMANTPUR,AVENUE SUPER MARTS LTD,Modern Trade,BIL1900317,...,WK01,CS0340EGW01R,CHIK HFP EGG WHITE PROT SH 340ML 24P,CHIK SHAMPOO EGG Bottle,48,"5,471.4240","5,723.5836",0.0572,Keys-Ac/s,Keys-Ac/s
1,MT0600032000290003200029,3200029,VENKATA SAI SALES CORPORATION,Key A/c Distributor,48.0000,RET0048,AVENUE SUPERMARTS LTD RAMANTPUR,AVENUE SUPER MARTS LTD,Modern Trade,BIL1900317,...,WK01,FCAM0050NAT03R,FAIREVER NAT FAIRNESS SOLN ND 50G 72PCS,FAIREVER Natural,96,"6,719.7700","5,980.8232",0.0598,Keys-Ac/s,Keys-Ac/s


In [92]:
data9.shape

(1126587, 31)

# Missing Prodcode in Product Hierarchy

In [90]:
data9 = pd.merge(data9, product_hierarchy, left_on = 'PrdCode', right_on = 'PrdCode',how='left')
missing_brand = data9[data9.Brand.isnull()][["PrdCode","Brand"]].drop_duplicates()
len(missing_brand)
if len(missing_brand) > 0:
    missing_brand.to_csv("D:\Analytics\Marketing Report\FY 19-20\Master Data\missing_brand.csv",index= False)

# Missing Category

In [91]:
data9 = pd.merge(data9,static_category_lookup,how='left',left_on = "RtrGroupCde",right_on="Old_Category") 
missing_category = data9[data9.Old_Category.isnull()]["RtrGroupCde"].to_frame().drop_duplicates()
len(missing_category)
if len(missing_category) > 0:
    missing_brand.to_csv("D:\Analytics\Marketing Report\FY 19-20\Master Data\missing_category.csv",index= False)

### Data Cleaning Function

1. Changing the column names to lower case.    
2. null_cols variable will show which column has the null values. Only taxable amount can have null values.
3. Imputing the NA values in category as <b>"Kirana"</b>
4. And Imputing the <b>rtrcode</b> and <b>rtrname</b> NA values as <b>"Data not found"</b>
5. Merging the data with <b>Master JC  cycle</b> created and finding the JC value for that data.
6. After Merging the columns used are
                distcode
                rtrcode
                prdcode
                cusname
                cusgroname
                billno
                invdate
                totqty
                basevalue
                div
                Jc
                Week
        
                

        
    

In [ ]:
def datacleaning(dataframe):
    data2               = dataframe
    data2.columns       = map(str.lower, data2.columns)
    data2               = data2[['statename', 'distcode', 'cusname', 'cusgroname', 'rtrid', 'rtrcode',
                                   'rtrname', 'billno', 'invdate', 'brand', 'b.pack', 'prdcode', 'prdname',
                                   'totqty', 'grossamt', 'taxableamt', 'netamt', 'basevalue', 'category',
                                   'div']]
    data2["rtrid"]      = data2["rtrid"].fillna(value = "Data not found") #New line
    data2["taxableamt"] = data2["taxableamt"].fillna(value = 0)
    data2["category"]   = data2["category"].fillna(value = "Kirana")
    data2[["rtrcode","rtrname"]]    = data2[["rtrcode","rtrname"]].fillna(value = "Data not found")
    null_data           = data2[data2.invdate.isnull()]
    if len(null_data) > 0 :
        print("Populate the null_data and append in the original data and proceed")
    null_cols           = data2.isnull().sum().to_frame().reset_index()
#     null_cols[null_cols[0] != 0]
    data2               = pd.merge(data2, jccycle, left_on = "invdate", right_on = "Wk_Start_Date")
    data2.drop(columns  = ["Wk_Start_Date"] ,axis = 1,inplace = True)
    data2[["rtrcode","category"]] = data2[["rtrcode","category"]].applymap(lambda s:(s.upper() if type(s)==str else s))
    print (null_cols)
    print(" \n If the above values are zero proceed to next step")

    return data2

In [99]:
data9.columns

Index(['distcode', 'rs code', 'distname', 'disttype', 'rtrid', 'rtrcode',
       'rtrname', 'rtrgroupcde', 'ret grp', 'billno', 'invdate', 'period',
       'prdcode', 'prdname_x', 'brand cat', 'totqty', 'netamt', 'basevalue',
       'in lakhs', 'region', 'zone', 'prdname_y', 'brand', 'rb', 'channel',
       'div', 'brdcat', 'brdpk', 'powersku', 'old_category', 'new_category'],
      dtype='object')

In [96]:
data9.ZONE.unique()

array(['Keys-Ac/s', 'E-Comm', 'Delhi', 'UP', 'Maharashtra', 'Gujarat',
       'BENGAL', 'JOB C', 'TN', 'KAR', 'AP', 'KERALA'], dtype=object)

In [97]:
data9.columns = map(str.lower, data9.columns)

In [106]:
data9[data9.rtrid.isnull()]["rtrid"]

Series([], Name: rtrid, dtype: object)

In [111]:
data9[data9.rtrid != "Data not found" & data9.rtrcode.isnull()][["rtrid","rtrcode"]]

TypeError: Cannot perform 'rand_' with a dtyped [bool] array and scalar of type [bool]

In [15]:
data9 = data9.rename(columns = {"distname" : "cusname",
                              "disttype" : "cusgroname",
                              "rtrgroupcde" : "category",
                               "brdpk" : "b.pack"
                               })

In [38]:
data9.columns

Index(['Distcode', 'RS CODE', 'DistName', 'DistType', 'Rtrid', 'RtrCode',
       'RtrName', 'RtrGroupCde', 'RET GRP', 'BillNo', 'InvDate', 'PERIOD',
       'PrdCode', 'PrdName_x', 'BRAND CAT', 'TotQty', 'NetAmt', 'BaseValue',
       'In Lakhs', 'REGION', 'ZONE', 'PrdName_y', 'Brand', 'RB', 'Channel',
       'Div', 'BrdCat', 'BrdPk', 'PowerSKU'],
      dtype='object')

In [29]:
product_hierarchy.columns

Index(['PrdCode', 'PrdName', 'Brand', 'RB', 'Channel', 'Div', 'BrdCat',
       'BrdPk', 'PowerSKU'],
      dtype='object')

In [ ]:
data = data.rename(columns = {"brdpk" : "b.pack",
                              "disttype" : "cusgroname",
                              "rtrgroupcde" : "category"})

In [17]:
data9 = data9[['statename', 'distcode', 'cusname', 'cusgroname', 'rtrid', 'rtrcode',
       'rtrname', 'billno', 'invdate', 'brand', 'b.pack', 'prdcode', 'prdname',
       'totqty', 'grossamt', 'taxableamt', 'netamt', 'basevalue', 'category',
       'div']]

KeyError: "['div', 'taxableamt', 'grossamt', 'brand', 'statename', 'b.pack'] not in index"